Analysis of HeroesOfPymoli

Observations:
The top per-person spending was by the 'Other / Non-Disclosed' followed closely by the 'Female' group.
The age group that spent the most was the 20-24 range, but the 35-39 year olds spent the most per person.
The 3 items that generated the most money were Oathbreaker, Nirvana, and Fiery Glass Crusader.

Recommendations:
While the male group was the largest and generated the most revenue, it is important to note that the other two gender groups generated more per person. It might be beneficial to dive deeper into the data and find out what those groups purchased. There could be an opportunity to target them with specific items.

A similar recommendation could be made for the 35-39 year old group. Find out what they're buying and compare to what the other top per-person spending groups are buying.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data_df = pd.read_csv(file_to_load)
purchase_data_df.head(5)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
purchase_data_df.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

In [3]:
##### Player Count

#Print total number of players
#Had to create a new data frame using the result of the calculation in uniquePlayer

uniquePlayer = purchase_data_df["SN"].nunique()
totalPlayers_df = pd.DataFrame([uniquePlayer],columns =["Total Players"])
totalPlayers_df

,Total Players
0,576


In [4]:
##### Purchasing Analysis (Total)

#run basic calculations to obtain number of unique items, average price, etc.
#Create a summary data frame to hold the results
#Optional: give the displayed data cleaner formatting
#Display the summary data frame


#Performed each calculation
#Grabbed a count of unique items
uniqueItem = purchase_data_df["Item Name"].nunique()

#Calculated the avg price and then formated
averagePrice = purchase_data_df["Price"].mean()
averagePrice = '${:,.2f}'.format(averagePrice)

#Ran a count of purchases to find the total # of sales transactions
purchaseCount = purchase_data_df["Price"].count()

#Found total revenue and reformatted
purchaseRevenue = purchase_data_df["Price"].sum()
purchaseRevenue = '${:,.2f}'.format(purchaseRevenue)

#Slapped it all together in a dataframe
purchase_analysis_df = pd.DataFrame([uniqueItem],columns = ["Number of Unique Items"]) 
purchase_analysis_df["Average Price"] = pd.DataFrame([averagePrice])
purchase_analysis_df["Number of Purchases"] = pd.DataFrame([purchaseCount])
purchase_analysis_df["Total Revenue"] = pd.DataFrame([purchaseRevenue])

purchase_analysis_df
#One issue I ran into was Unique Items - the example output had 183
#but I came up with 179. I ran a pivot table against the data and
#came up with 179. Not sure where the variance is on this one.

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [5]:
##### Gender Demographics

#This one took a while to figure out, and it's pretty simple
#created a new df with just gender and sn
#found the drop_duplicates function (yay!) - this was the key
#grabbed the value counts of gender
#to find the percent I divided the unique gender counts by the total "uniquePlayer"
#that I had used above to build the unique player count

####NEW DATAFRAME
gender_group_df = pd.DataFrame(purchase_data_df[['Gender','SN']])

####REMOVE DUPLICATES
unqiue_gender_sn = gender_group_df.drop_duplicates(['SN', 'Gender'])

####WITH DUPLICATES GONE, DO GENDER COUNT
unique_counts_df = unqiue_gender_sn['Gender'].value_counts()

####FIND PERCENT AND ROUND AT THE SAME TIME
player_percent = (unique_counts_df/uniquePlayer *100).round(2)

####SLAP IT ALL TOGETHER
totally_smoogadly = pd.DataFrame({"Total Count":unique_counts_df,
                                "Percentage of Players":player_percent})
####BOOM
totally_smoogadly

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [6]:
##### Purchasing Analysis (Gender)

#Following the previous theme - it all begins with a new dataframe
#Began with an empty dataframe this time
#while troubleshooting I found several of examples where people began with an empty dataframe
#http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html

###CREATE EMPTY DF
purchasing_analysis_df = pd.DataFrame()

###PERFORM GROUPBY
gender_purch_group_df = purchase_data_df.groupby(['Gender'])['Price']

###WITH GROUPING IN PLACE - USED SIZE FUNCTION TO GET OUR FIRST COLUMN
###ADDS THE COLUMN TO THE DF
purchasing_analysis_df['Purchase Count'] = gender_purch_group_df.size()

###FINDS AVG PURCHASE PRICE, FORMATS TO MONIES, ADDS TO DF
purchasing_analysis_df['Average Purchase Price'] = gender_purch_group_df.mean().round(2).map("${:,.2f}".format)

###FINDS TOT PURCH VAL, FORMATS TO MONIES
purchasing_analysis_df['Total Purchase Value'] = gender_purch_group_df.sum().map("${:,.2f}".format)

###PSEUDO-PLAGARIZED A BIT ON THIS ONE TO MAKE IT WORK
#HAD A HELLUVA TIME GETTING THIS ONE GOING
#DUG AROUND FOREVER AND FOUND WHERE SOMEONE HAD A SIMILAR PROBLEM - HEY, NEW DF...UGH
#IT WORKS, BUT THERE IS GUILT HERE (AT LEAST MORE THAN USUAL)

###CREATES A NEW DF, GROUPING SN AND GENDER - AND ADDS A PRICE COLUMN THAT IS A SUM OF THE GROUPING
unique_df = purchase_data_df.groupby(['SN', 'Gender'])['Price'].sum()

#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html
###WILL GET A COLUMN ERROR IF YOU DON'T RESET THE INDEX
unique_df = unique_df.reset_index()

###CALCULATES AVG PURCH PER PERSON, CONVERTS TO MONIES, AND ADDS TO DF
purchasing_analysis_df['Avg Total Purchase Per Person'] = unique_df.groupby(['Gender'])['Price'].mean().map("${:,.2f}".format)

purchasing_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
##### Age Demographics
#Setting up BINs
age_range_bin = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 100]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#this line is magical
pd.options.display.float_format = '{:,.2f}'.format

#copying DF
group_bin_df = purchase_data_df.copy()

#drop it like it's hot, or whatever the kids say
group_bin_df.drop_duplicates(['SN', 'Gender'], inplace=True, keep='first')

#Adding the age range to the DF
group_bin_df['Age_range'] = pd.cut(group_bin_df['Age'], age_range_bin, labels=group_names)

#another new DF
age_demo_df = pd.DataFrame()

#Setting columns and calculations
age_demo_df['Total Count'] = group_bin_df.groupby(['Age_range'])['Age'].count()
age_demo_df['Percentage of Players']=(group_bin_df.groupby(['Age_range'])['Age'].count() * 100)/float(uniquePlayer)

#del df.index.name # Remove the groupby generated axis name
age_demo_df

,Total Count,Percentage of Players
Age_range,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [8]:
##### Purchasing Analysis (Age)

#Setting up BINs
age_range_bin = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 100]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#This amazing line of code hooks us up with fantastic formatting the rest of the assignment
pd.options.display.float_format = '${:,.2f}'.format

#New DF, copying the old
group_bin_df = purchase_data_df.copy()

#Ever squeeze a potato? Like really SQUEEZE a potato?
#This is nothing like that
#But this does add our age ranges to the copied DF
#Which comes in useful for grouping and calculating
group_bin_df['Age_range'] = pd.cut(group_bin_df['Age'], age_range_bin, labels=group_names)

#Creating a blank DF
purch_analysis_df = pd.DataFrame()

#Performing calculations and adding to the new DF
purch_analysis_df['Purchase Count'] = group_bin_df.groupby(['Age_range'])['Age'].count()
purch_analysis_df['Average Purchase Price'] = group_bin_df.groupby(['Age_range'])['Price'].mean()
purch_analysis_df['Total Purchase Value'] = group_bin_df.groupby(['Age_range'])['Price'].sum()

#Still owrking outthe maths on this one
purch_analysis_df['Avg Total Purchase per Person'] = group_bin_df.groupby(['Age_range'])['Price'].mean()
#df['Avg Total Purchase per Person']=(group_bin_df.groupby(['Age_range'])['Age'].count() * 100)/float(uniquePlayer)

#del df.index.name # Remove the groupby generated axis name
purch_analysis_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age_range,,,,
<10,23,$3.35,$77.13,$3.35
10-14,28,$2.96,$82.78,$2.96
15-19,136,$3.04,$412.89,$3.04
20-24,365,$3.05,"$1,114.06",$3.05
25-29,101,$2.90,$293.00,$2.90
30-34,73,$2.93,$214.00,$2.93
35-39,41,$3.60,$147.67,$3.60
40+,13,$2.94,$38.24,$2.94


In [9]:
###### Top Spenders
####NEW DATAFRAME
top_spenders_df = pd.DataFrame()

#setting up the groupby and pulling in pricing
sn_spender_df = purchase_data_df.groupby(['SN'])['Price']

#Using the grouping, making calculations, adding to empty DF
top_spenders_df['Purchase Count'] = sn_spender_df.size()
top_spenders_df['Average Purhcase Price'] = sn_spender_df.mean().round(2)
top_spenders_df['Total Purchase Value'] = sn_spender_df.sum()
top_spenders_df = top_spenders_df.sort_values(by='Total Purchase Value', ascending=False)

top_spenders_df.head(5)

,Purchase Count,Average Purhcase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [10]:
##### Most Popular Items


####NEW DATAFRAME
pop_items_df = pd.DataFrame()

#Grouping and bringing in price
itemid_df = purchase_data_df.groupby(['Item ID','Item Name'])['Price']

#Using grouping to make calculations and add to the empty DF
pop_items_df['Purchase Count'] = itemid_df.size()
pop_items_df['Item Price'] = itemid_df.mean().round(2)
pop_items_df['Total Purchase Value'] = itemid_df.sum()

#Sorting
pop_items_df = pop_items_df.sort_values(by='Purchase Count', ascending=False)

pop_items_df.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [11]:
##### Most Profitable Items
#Changing the sort
pop_items_df = pop_items_df.sort_values(by='Total Purchase Value', ascending=False)
pop_items_df.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
